# Integrative analysis of pathway deregulation in obesity #

## Python implementation

### Steps (according to the paper)

1. Probes containing missing values are excluded from the analysis. 

2. Probes are mapped to Entrez ID labels if they are available in the associated platform. Otherwise the David portal is used to convert the available labels to Entrez ID labels. 

3. Values corresponding to raw expression counts or gene expression intensity are log2 transformed (if necessary). 

4. Probes mapping to the same Entrez ID label are averaged out. 

5. Probes that cannot be mapped to a unique Entrez ID label are excluded from the analysis, as well as those that cannot be mapped to any Entrez ID label at all. 

6. We apply a simple L1 normalization in linear space, imposing that the sum of expression of all genes is constant among samples. After these steps, each data set or batch is represented by a single expression matrix X. Each entry Xi j represents the log2 of the expression intensity of gene i in sample j.

### Imports

In [1]:
# Import std libraries
import os
from operator import itemgetter 
import re

# Import third party
import numpy as np
import pandas as pd
import GEOparse

# Set logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logging.getLogger("GEOparse").setLevel(logging.WARNING)

### Load Dataset

Download the dataset (if needed) and load it.

Some GEOparse names:
- DataSet (GDS)
- Series (GSE)
- Platform (GPL)
- Samples (GSM)

In [2]:
def load_dataset(dataset_id):
    """
    Load the dataset from disk (or download it if it does not exists)
    Arguments:
    - dataset_id: the ID of the dataset to load
    
    Output:
    - GSE object (GEOparse Series)
    """
    path = "./" + dataset_id + "_family.soft.gz"
    if os.path.exists(path):
        # Load from an existing file
        print("- Loading from", path)
        gse = GEOparse.get_GEO(filepath=path)
    else:
        # Download GSE and load it
        print("- Downloading", dataset_id)
        gse = GEOparse.get_GEO(geo=dataset_id, destdir="./")
    return gse

dataset_id = "GSE48964"
gse = load_dataset(dataset_id)
print("- Dataset loaded")

- Downloading GSE48964
D: 100% - 12.8MiB  / 12.8MiB  eta 0:00:00
- Dataset loaded


### Get Info

Get some useful info and statistics from our data.

We're going to extract:
- number of platforms
- number of samples
- dimension of each sample

In [3]:
# data_frames contains the data-frame of each sample
# samples_name is a list which contains the name associated to each dataframe
data_frames = []
samples_information = []

#this variable is not used but will help in finding elements that are correlated in the original experiment
association_dictio = {}

for gsm_name, gsm in gse.gsms.items():
    #print(gsm.metadata)
    
    if "obese" in gsm.metadata['title'][0]:
        identifier = gsm.metadata['geo_accession'][0] + "_OU"
    elif "lean" in gsm.metadata['title'][0]:
        identifier = gsm.metadata['geo_accession'][0] + "_LU"
    
    df = gsm.table
    df['ID_REF'] = df['ID_REF'].map(str)
    data_frames.append(df)
    samples_information.append((gsm.metadata['geo_accession'][0], identifier))


print("- Number of platforms", len(gse.gpls.items()))
print("- Number of samples", len(data_frames), "Remember, all Female in this dataset")
print("- Dimension of each sample (assuming are all the same)", data_frames[0].shape)
print('\nExample of a sample dataframe:')
print(data_frames[0].head())
print(data_frames[0].shape)

- Number of platforms 1
- Number of samples 6 Remember, all Female in this dataset
- Dimension of each sample (assuming are all the same) (33297, 2)

Example of a sample dataframe:
    ID_REF    VALUE
0  7892501  4.97979
1  7892502  4.95171
2  7892503  6.82531
3  7892504  8.53222
4  7892505  5.47718
(33297, 2)


### Filter data #1

We're going to:
- Remove probes from the mapper, mapping to multiple Entrez IDs
- Construct a Python dictionary containing the valid probes and their Entrez ID

In [4]:
def create_mapper(meta_data_tables):
    """
    Returns a python dictionary that represents our mapper object
    Important: not all probs_id are mapped to an ENTREZ_GENE_ID
    probs_id without an enterez_id are not added to the dictionary
    """
    mapper = {}
    for row in meta_data_tables.iterrows():
        #print(row)
        #print(type(row[1]), len(row))
        
        if str(row[1][1]) == 'nan':
            continue
        
        probs_id = row[1][0]

        if probs_id in mapper and mapper[probs_id] != row[1][1]:
            # Multiple enterez id for the same probs
            # Set their value to None to invalid them
            # Elements set to "None" are then removed
            mapper[probs_id] = None

        if probs_id not in mapper and not pd.isnull(row[1][1]):
            mapper[probs_id] = row[1][1]
            
    # Remove invalid mapping (value = None)
    # (Some of the probes are linked with multiple numbers (enterez_id ?) using /// as separator)
    filtered_mapper = {k:v for k,v in mapper.items() if v != None and '/' not in v}
    
    return filtered_mapper

meta_data_tables = pd.read_csv("./data/GPL6244.annot", sep="\t", skiprows=27, usecols=["ID","Gene ID"], low_memory=True, dtype=str)
#print(df.head())
mapper = create_mapper(meta_data_tables)
print("- Mapper loaded", len(mapper))

- Mapper loaded 20053


### Filter data #2: 
#### Remove rows without a matching enterez_id
Intuition: 
1. Convert the dictionary **mapper** into a pandas' DataFrame (**mapper_df**).  
2. Use a SQL-like inner join to merge **mapper_df** with the existing pandas' DataFrame.  
Inner join creates a new Dataframe with *only* the matching rows.

References:
- https://www.w3schools.com/sql/sql_join_inner.asp
- https://pandas.pydata.org/pandas-docs/stable/merging.html

In [5]:
# Convert mapper to a Pandas Dataframe with two columns (probs, enterez_id)
mapper_df = pd.DataFrame.from_dict(mapper, orient='index')
mapper_df.index.name = 'ID_REF'
mapper_df.columns = ['ENTREZ_GENE_ID']
mapper_df['ENTREZ_GENE_ID'] = mapper_df['ENTREZ_GENE_ID'].map(str)
print(type(mapper_df['ENTREZ_GENE_ID'][0]))
mapper_df.index.map(str)

print("\nExample of the obtained Pandas DataFrame:")
print(mapper_df.head())

# Create a mapper (sample_id, person_id)
#mapper_sample_person = pd.DataFrame(samples_label)
#mapper_sample_person = mapper_sample_person.set_index([samples_name])
#mapper_sample_person = mapper_sample_person.transpose()
#mapper_sample_person.head()

'1278' in mapper.values()

<class 'str'>

Example of the obtained Pandas DataFrame:
        ENTREZ_GENE_ID
ID_REF                
7977841          54930
8127109          22858
7946033           3043
8144213           7434
8168303           6872


True

### Creation of a Dataframe Entrez ID - Value: genes common to all array
For each sample:
1. We convert probes' values in **log2(values)**
2. Rows with the same **entrez_id** are merged together using the average (probes mapping to the same Entrez ID are averaged out).

In [6]:
# Update each df object ainside the list 'data_frames' with the dataframe with only matching probs_id
# For each sample's dataframes:
# - log2
# - only matching entrez_id
# - L1 normalization

# - WARNING in this dataset the inner merge doesn't work and so we need to manually eliminate the ID_REF column

data_frames_entrez = []
for df in data_frames:
    df["ID_REF"] = df["ID_REF"].map(str)
    mdf = pd.merge(df, mapper_df, how='inner', left_on=['ID_REF'], right_index=True, sort=False)
    mdf = mdf.groupby('ENTREZ_GENE_ID').mean()
    mdf['VALUE'] = 2**mdf['VALUE']
    mdf.VALUE = mdf.VALUE / mdf.VALUE.sum()
    mdf['VALUE'] = np.log2(mdf['VALUE'])
    data_frames_entrez.append(mdf)

# concat data_frames by columns and use the sample GSM identifier as axis
# concat data_frames by columns and use the sample GSM identifier as axis
merged_entrez_value_df = pd.concat(data_frames_entrez, axis=1, keys=samples_information)
print("size before any removal", merged_entrez_value_df.shape)

# remove gene with at least 1 missing value
merged_entrez_value_df = merged_entrez_value_df.dropna(axis=0, how='any')
print("size after removing genes with missing value", merged_entrez_value_df.shape)

# remove disturbing index "VALUE"
merged_entrez_value_df.columns = merged_entrez_value_df.columns.droplevel([0, 2])


final_df = merged_entrez_value_df

# Change index type from numpy.int64 to str
final_df.index = final_df.index.map(str)

final_df.to_pickle("data/GSE48964_table.pkl")

print("\nExample of the obtained merged data frame:")
merged_entrez_value_df.head()

size before any removal (19037, 6)
size after removing genes with missing value (19037, 6)

Example of the obtained merged data frame:


,GSM1187675_OU,GSM1187673_OU,GSM1187677_LU,GSM1187674_OU,GSM1187676_LU,GSM1187678_LU
ENTREZ_GENE_ID,,,,,,
1,-14.290179,-14.238613,-14.264758,-14.213429,-14.254831,-14.271377
10,-15.817946,-15.832923,-15.682516,-15.601905,-15.767085,-15.665861
100,-13.640160,-13.677735,-13.530120,-13.780287,-13.605710,-13.734928
1000,-13.080175,-12.995080,-13.034852,-13.032072,-13.588164,-13.020116
10000,-13.620189,-13.545918,-13.601167,-13.596763,-13.638799,-13.620340
